## Necassary imports

In [3]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import ResNet50
from keras.applications.resnet import preprocess_input

<a id='item32'></a>


## Defining Global Variables

In [6]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

<a id='item34'></a>


## Preprocessing Data

In [7]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [8]:
train_generator = data_generator.flow_from_directory(
    '../input/concrete_data/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 29958 images belonging to 2 classes.


In [9]:
validation_generator = data_generator.flow_from_directory(
    '../input/concrete_data/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 10042 images belonging to 2 classes.


<a id='item35'></a>


## Creating the Model

In [10]:
model = Sequential()

In [11]:
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

In [12]:
model.add(Dense(num_classes, activation='softmax'))

In [13]:
model.layers

In [14]:
model.layers[0].layers

In [15]:
model.layers[0].trainable = False # turn off training for the ResNet50 layers

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


### Defining Model metrics

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
steps_per_epoch_training = len(train_generator) # total number of samples in training set / batch_size_training = 29845/100 = 300
steps_per_epoch_validation = len(validation_generator) # total number of samples in validation set / batch_size_validation = 10050/100 = 100
num_epochs = 2

## Training Model

In [19]:
# fit_generator as we use ImageDataGenerator that generates batches of images indefinitely (divide the dataset into batches)
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

C:\Users\ahmed\AppData\Local\Temp\ipykernel_14264\387289610.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history = model.fit_generator(


Epoch 1/2
300/300 [==============================] - 1326s 4s/step - loss: 0.0290 - accuracy: 0.9906 - val_loss: 0.0081 - val_accuracy: 0.9982
Epoch 2/2
300/300 [==============================] - 1301s 4s/step - loss: 0.0061 - accuracy: 0.9985 - val_loss: 0.0046 - val_accuracy: 0.9992


## Saving the Model

In [20]:
model.save('../models/classifier_resnet_model.h5')